In [ ]:
from sensor_core import SensorManager
from sensor_core import strg_manager
from sensor_core import ser_manager
import numpy as np

In [ ]:
# Find available serial ports
ser_manager.find_serial()

In [ ]:
# USER DEFINED PARAMS
channel_key = ["time", "PV0", "PV1", "PV2", "PV3", "roll", "pitch", "yaw"]  # Names of channels
commport = '/dev/ttyACM0'                    # Name of comm port
baudrate = 115200                            # bits/second
window_size = 1                              # Number of points to acquire before updating plot

In [ ]:
# Custom Serial Acquisition Function
def acquire_data(ser, window_size, num_channel):
    """ serial acquisition function handler. All parameters are internally generated
    Must include the following parameters:
    :param ser: serial port
    :param window_size: number of frames to capture before passing to shared memory object. Default 1
    :param num_channel: number of channels to capture. 
    """
    ser_data = np.zeros(num_channel)
    channel_data = np.array([])

    try:
        data = ser.readline().decode('utf-8').strip().split('\t')
        for i in range(np.shape(data)[0]):
            ser_data[i] = float(data[i])
    except:
        pass

    channel_data = np.append(channel_data, ser_data)
    return channel_data

In [ ]:
# Test custom serial function
# ser = ser_manager.SerialManager(commport=commport, baudrate=baudrate, num_channel=8)
# ser.setup_serial()
# ser.acquire_data(func=acquire_data)

In [ ]:
# Instantiate Sensor Manager class
sm = SensorManager(channel_key=channel_key, commport=commport, 
                   baudrate=baudrate, num_points=100, dtype=np.float64)

In [ ]:
# Create 'update process' to handle acquired data
p1 = sm.update_data_process(save_data=True, 
                            filepath = "./serial_db.sqlite3",
                            func=acquire_data)
plot = sm.setup_plot()

In [ ]:
# Start update process, start plotting process
## Recommended to use in-built start_process function to avoid Windows OS issues
sm.start_process(p1)
plot.add_animations(sm.online_grid_plot_data)

In [ ]:
# Show plot
plot.show()